# Test with Jupyter

In [3]:
from math import sqrt, atan, cos, sin
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from transfer_functions import EOM_particle_in_Bfield, PtoV, EtoP

import pandas as pd
import time
start_time = time.clock()


plt.close('all')


path = "D:/beamline/magnets/field maps/test/"
field_map = pd.read_csv(path + "test.csv")
spatial_res = 10


# replacing blank spaces with '_'  
field_map.columns =[column.replace(" ", "_") for column in field_map.columns] 
field_map.columns =[column.replace("[", "") for column in field_map.columns] 
field_map.columns =[column.replace("]", "") for column in field_map.columns] 

#print(field_map[['z_[mm]']])

x = 0
y =0
z = 50


temp_df = field_map.loc[(field_map['x_mm'] >= x - spatial_res) & (field_map['x_mm'] <= x + spatial_res)
                        & (field_map['y_mm'] >= y - spatial_res) & (field_map['y_mm'] <= y + spatial_res) 
                        & (field_map['z_mm'] >= z - spatial_res) & (field_map['z_mm'] <= z + spatial_res)]
temp_df['distXY'] = (temp_df['x_mm'] - x)**2 + (temp_df['y_mm'] - y)**2 
temp_df['dist'] = (temp_df['x_mm'] - x)**2 + (temp_df['y_mm'] - y)**2 + (temp_df['z_mm'] - z)**2 
temp_df = temp_df.sort_values(['dist','distXY']) 


[Bx,By,Bz] = temp_df.iloc[0].loc[['Bx_T','By_T','Bz_T']]



nb_part=1

nb_it_z = 300
Delta_z = 0.001


beam = np.empty(shape=[nb_it_z,nb_part,6]) 

beam_ref = np.empty(shape=[nb_it_z,6]) 
costheta_mat = np.empty(shape=[nb_it_z]) 
sintheta_mat = np.empty(shape=[nb_it_z]) 


refE = 150
refp = EtoP(refE)
ref_speed = PtoV(refp,938)


# trajectory of reference particle
for it_z in range(0,nb_it_z):
        
    Delta_t = Delta_z/ref_speed
    
    By = 1
    
    [ref_x,ref_y,ref_z,ref_vx,ref_vy,ref_vz] = EOM_particle_in_Bfield(Delta_t,0,0,0,0,0,ref_speed,0,By)
    beam_ref[it_z,:] = [ref_x,ref_y,ref_z,ref_vx,ref_vy,ref_vz]
    #theta_ZX = -atan(ref_vx/ref_vz)
    costheta_mat[it_z] = 1/sqrt(1+(ref_vx/ref_vz)**2)
    sintheta_mat[it_z] = ref_vx/ref_vz/sqrt(1+(ref_vx/ref_vz)**2)
    


for it_z in range(0,nb_it_z):

    z = ref_speed*Delta_t*it_z
    
    
    for it_p in range(0,nb_part):
        
        if it_z == 0: #initial conditions
            x=0
            y=0
            z=0
            vx = 0
            vy = 0
            vz = 1.5*10**8
            v0 = vz
            beam[it_p,0,:] = [x,y,z,vx,vy,vz]


        # extract subset around region of interest
        temp_df = field_map.loc[
            (field_map['x_mm'] >= x - spatial_res) & (field_map['x_mm'] <= x + spatial_res) & 
            (field_map['y_mm'] >= y - spatial_res) & (field_map['y_mm'] <= y + spatial_res) & 
            (field_map['z_mm'] >= z - spatial_res) & (field_map['z_mm'] <= z + spatial_res)]
        # compute distance to point of interest
        temp_df['distXY'] = (temp_df['x_mm'] - x)**2 + (temp_df['y_mm'] - y)**2 
        temp_df['dist'] = (temp_df['x_mm'] - x)**2 + (temp_df['y_mm'] - y)**2 + (temp_df['z_mm'] - z)**2 
        temp_df = temp_df.sort_values(['dist','distXY']) 
        # get field at closest point
        [Bx,By,Bz] = temp_df.iloc[0].loc[['Bx_T','By_T','Bz_T']]
        
        
        
        Delta_t = Delta_z/vz
        [x,y,z,vx,vy,vz] = EOM_particle_in_Bfield(Delta_t,x,y,z,vx,vy,vz,Bx,By,Bz)
        
        
        # compute difference with reference particle
        x = x - beam_ref[it_z,0]
        y = y - beam_ref[it_z,1]
        
        vx = costheta_mat[it_z]*vx - sintheta_mat[it_z]*vz
        vz = sintheta_mat[it_z]*vx + costheta_mat[it_z]*vz
        
        # normalize to correct rounding errors
        v = sqrt(vx**2 + vy**2 + vz**2)
        vx = vx * v0/v
        vy = vy * v0/v
        vz = vz * v0/v
        
        
        beam[it_z,it_p,:] = [x,y,z,vx,vy,vz]
    

print("--- execution time: %s seconds ---" % (time.clock() - start_time))

plt.figure(0)
plt.plot(beam[:,0,2],beam[:,:,0])
plt.figure(1)
plt.plot(beam[:,0,2],beam[:,:,1]) 



C:\Users\HILAM\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  if __name__ == '__main__':
C:\Users\HILAM\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\HILAM\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\HILAM\Anaconda3\lib\site-pack

--- execution time: 6.9199261999999635 seconds ---


C:\Users\HILAM\Anaconda3\lib\site-packages\ipykernel_launcher.py:130: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [11]:
field_map

,x_mm,y_mm,z_mm,Bx_T,By_T,Bz_T
0,-50,-30,0,0.000190,1.000017,-0.000038
1,-48,-30,0,0.000068,0.999826,0.000062
2,-46,-30,0,0.000170,0.999877,0.000080
3,-44,-30,0,-0.000063,0.999930,0.000175
4,-42,-30,0,-0.000137,0.999963,0.000054
...,...,...,...,...,...,...
224995,40,28,298,-0.000147,1.000103,-0.000081
224996,42,28,298,-0.000206,0.999875,-0.000065
224997,44,28,298,-0.000034,0.999954,-0.000073
224998,46,28,298,0.000080,0.999932,-0.000023


In [12]:
temp_df

,x_mm,y_mm,z_mm,Bx_T,By_T,Bz_T,distXY,dist
775,0,0,0,-0.000070,0.999940,-8.057534e-08,1.714346e-07,0.089401
2275,0,0,2,0.000090,0.999919,-2.474256e-04,1.714346e-07,2.893402
774,-2,0,0,0.000094,1.000090,8.234195e-05,3.998344e+00,4.087745
725,0,-2,0,-0.000003,1.000073,2.674422e-05,3.999993e+00,4.089394
825,0,2,0,0.000041,1.000051,4.392164e-05,4.000007e+00,4.089408
...,...,...,...,...,...,...,...,...
8470,-10,8,10,0.000077,1.000056,1.315281e-04,1.639917e+02,258.101154
8021,-8,-10,10,-0.000065,0.999952,-2.271862e-05,1.639933e+02,258.102747
8029,8,-10,10,0.000264,0.999895,1.126188e-04,1.640066e+02,258.115996
6520,-10,-10,8,0.000034,1.000101,3.248133e-05,1.999917e+02,259.297090


In [15]:
from numba.typed import List
from numba import njit

@njit
def boolean_index_numba_multiple(array, xmin, xmax, ymin, ymax, zmin, zmax):
    """
    Takes a numpy array and isolates all points that are within [xmin, xmax]
    for the first dimension, between [ymin, ymax] for the second dimension
    and [zmin, zmax]  for the third dimension by creating a boolean index.
    This function will be compiled with numba.
    """
    index = ((array[:, 0] > xmin) & (array[:,  1] > ymin) & (array[:, 2] > zmin)
             & (array[:, 0] < xmax) & (array[:, 1] < ymax) & (array[:, 2] < zmax))

    return array[index]

In [4]:


test = np.searchsorted([-5,-4,-3,-2,-1,0,1,2,3,4,5,6,7,8,9], 1000, side="left")
test

15